In [1]:
import yaml
import duckdb
import psycopg2
import pandas as pd

### Carregar config

In [2]:
def load_config(config_path="config.yaml"):
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
    return config

### Realizar Consulta

In [3]:
def query_postgres(config, query):
    conn = psycopg2.connect(
        host=config['database']['host'],
        port=config['database']['port'],
        user=config['database']['user'],
        password=config['database']['password'],
        dbname=config['database']['dbname']
    )
    cur = conn.cursor()
    cur.execute(query)
    
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    
    cur.close()
    conn.close()
    
    # Converter para DataFrame
    df = pd.DataFrame(data, columns=colnames)
    return df

### Persistir dados da Consulta

In [4]:
def save_to_duckdb(duckdb_conn, df, table_name):
    duckdb_conn.register('df_view', df)
    duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df_view")

### Executar

In [5]:
config = load_config()
    
duckdb_conn = duckdb.connect("credito.duckdb")

queries = {
    'PedidoCredito': "SELECT * FROM public.PedidoCredito;",
    'ParcelasCredito': "SELECT * FROM public.ParcelasCredito;",
    'ProdutosFinanciados': "SELECT * FROM public.ProdutosFinanciados;"
}

for table_name, query in queries.items():
    try:
        df = query_postgres(config, query)
        save_to_duckdb(duckdb_conn, df, table_name)
        print(f"Tabela {table_name} carregada com sucesso.")
    except Exception as e:
        print(f"Erro ao carregar a tabela {table_name}: {e}")

duckdb_conn.close()

Tabela PedidoCredito carregada com sucesso.
Tabela ParcelasCredito carregada com sucesso.
Tabela ProdutosFinanciados carregada com sucesso.


### Testar Persistência

In [9]:
duckdb_conn = duckdb.connect("credito.duckdb")

df = duckdb_conn.execute(f"SELECT * FROM credito.ProdutosFinanciados").fetchdf()
duckdb_conn.close()
print(df)

   produtoid   nomecomercial  precotabela
0          1    AgileXplorer     250000.0
1          2    VoyageRoamer     350000.0
2          3     EcoPrestige     500000.0
3          4      WorkMaster     280000.0
4          5      DoubleDuty     320000.0
5          6       SpeedFury     800000.0
6          7  TrailConqueror     400000.0
7          8   ElegantCruise     300000.0


In [10]:
duckdb_conn = duckdb.connect("credito.duckdb")

df = duckdb_conn.execute(f"SELECT * FROM credito.ParcelasCredito").fetchdf()
duckdb_conn.close()
print(df)

      solicitacaoid  numeroparcela  valorparcela datavencimento datapagamento  \
0              1160              1       3636.42     2023-08-08           NaT   
1              1160              2       3636.42     2023-09-07    2023-09-07   
2              1160              3       3636.42     2023-10-07    2023-10-07   
3              1160              4       3636.42     2023-11-06    2023-11-06   
4              1160              5       3636.42     2023-12-06    2023-12-06   
...             ...            ...           ...            ...           ...   
3183           2102              1      20833.33     2023-08-02           NaT   
3184           2401              1      20833.33     2023-08-02           NaT   
3185           2935              1      20833.33     2023-08-02    2023-08-02   
3186           2313              1      20833.33     2023-08-02    2023-08-02   
3187           1399              1      20833.33     2023-08-02    2023-08-02   

       status  
0     Venci

In [11]:
duckdb_conn = duckdb.connect("credito.duckdb")

df = duckdb_conn.execute(f"SELECT * FROM credito.ProdutosFinanciados").fetchdf()
duckdb_conn.close()
print(df)

   produtoid   nomecomercial  precotabela
0          1    AgileXplorer     250000.0
1          2    VoyageRoamer     350000.0
2          3     EcoPrestige     500000.0
3          4      WorkMaster     280000.0
4          5      DoubleDuty     320000.0
5          6       SpeedFury     800000.0
6          7  TrailConqueror     400000.0
7          8   ElegantCruise     300000.0
